In [ ]:
from database import get_dataframe_from_sql_query
from datetime import datetime as dt, timedelta as td
from constant import BEEXP_SCRIPT, ORDER_ITEM_DATE_COLS
from messages import send_messages_by_group
first_update = dt.now() - td(days=7)
current_update = first_update
last_update = first_update
def get_beexp_since(ordered_date):
    oders_df = get_dataframe_from_sql_query(
    sql_script=BEEXP_SCRIPT,
    date_cols=ORDER_ITEM_DATE_COLS
    )
    mask = (oders_df['dt_implant'] >= ordered_date)
    return oders_df.loc[mask]

beexp_df = get_beexp_since(current_update)
products = [f"Quantidade de Ingressos: [dado] Nome do Vendedor: [dado] Tipo de Ingresso: [dado] Valor faturado: [dado] Forma de pagamento: [dado] Código do cliente: [dado] Data da venda: [dado] E-mail do representante: [dado] O respectivo vendedor já recebeu o link de cadastro do participante." for cod in ]
send_messages_by_group(
        message_body=products, group='comercial'
)

In [ ]:
script = 'data/in/beexp_orders.sql'
from database import get_dataframe_from_sql
beexp_df = get_dataframe_from_sql(script)     
beexp_filtered_df = beexp_df[['qtd', 'nome_colaborador', 'desc_comercial', 'preco_total', 'cond_pgto', 'cod_cliente', 'nome_cliente', 'dt_implant', 'email_colaborador']]

In [ ]:
from mkt_automation import get_beexp_since, get_beexp_manager_message
from datetime import datetime as dt, timedelta as td
from messages import send_messages_by_group
first_update = dt.now() - td(days=7)
current_update = first_update
last_update = first_update

beexp_df = get_beexp_since(current_update)
beexp_df = beexp_df[['qtd', 'nome_colaborador', 'desc_comercial', 'preco_total', 'cond_pgto', 'cod_cliente', 'nome_cliente', 'dt_implant', 'email_colaborador']]
for row in beexp_df.itertuples():
  beexp_message = get_beexp_manager_message(row)
  send_messages_by_group(message_body=beexp_message, group='beexp_manager')

In [ ]:
import polars as pl
sellers_df = pl.read_csv('data/in/vendedores.csv', separator=';')
sellers_df = sellers_df.select(
    pl.col('cod_colaborador'),
    pl.col('nome_colaborador'),
    pl.col('ddd_celular').cast(pl.Utf8).alias('ddd_celular'),
    pl.col('celular').cast(pl.Utf8).alias('celular'),
    pl.col('ddd_1').cast(pl.Utf8).alias('ddd_1'),
    pl.col('telefone_1').cast(pl.Utf8).alias('telefone_1'),
    pl.col('email')
)

In [ ]:
sellers_contacts = sellers_df.with_columns([
    pl.when((
        pl.col('ddd_celular').is_not_null()) & (pl.col('celular').is_not_null()))
    .then(pl.map(['ddd_celular', 'celular'], lambda s: '+55'+s[0]+s[1]))
    .otherwise(None)
    .alias('cellphone')    
])

sellers_contacts = sellers_contacts.with_columns([
    pl.when((
        pl.col('ddd_1').is_not_null()) & (pl.col('telefone_1').is_not_null()))
    .then(pl.map(['ddd_1', 'telefone_1'], lambda s: '+55'+s[0]+s[1]))
    .otherwise(None)
    .alias('phonenumber')
])

sellers_contacts = sellers_contacts.with_columns([
    pl.when(pl.col('cellphone').is_not_null())
    .then(pl.col('cellphone'))    
    .otherwise(None)
    .alias('phone')
])

sellers_contacts = sellers_contacts.with_columns([
    pl.when(pl.col('phone').is_not_null())
    .then(pl.col('phone'))    
    .otherwise(pl.col('phonenumber'))
    .alias('phone')
])

sellers_contacts = sellers_contacts.drop_nulls(['phone']).select(pl.col(['cod_colaborador', 'nome_colaborador','phone','email']))

sellers_contacts = sellers_contacts.with_columns([
    pl.when(pl.col('cod_colaborador').is_not_null())
    .then(pl.lit("['beexp_seller']"))    
    .otherwise(pl.lit("['beexp_seller']"))
    .alias('groups')
])

sellers_contacts.columns = ['uno_id', 'name', 'phone', 'email', 'groups']

In [ ]:
sellers_contacts.write_json('messages/sellers_contacts.json', row_oriented=True)

In [ ]:
sellers_contacts_bot_conversa = sellers_contacts.select(pl.col(['phone', 'name', 'uno_id']))
sellers_contacts_bot_conversa = sellers_contacts_bot_conversa.with_columns([
    pl.when(pl.col('uno_id').is_not_null())
    .then(pl.col('uno_id').cast(pl.Utf8).apply(lambda x: f"__uno__.sg_colaborador.{x}"))
    .otherwise(pl.lit(None))
    .alias('etiquetas')
]).select(pl.col(['phone', 'name', 'etiquetas']))
sellers_contacts_bot_conversa.columns = ['telefone', 'nome', 'etiquetas']

In [ ]:
sellers_contacts_bot_conversa.write_excel('data/out/vendedores_uno_botconversa.xlsx')

In [ ]:
script = 'data/in/contact_sellers.sql'
from database import get_dataframe_from_sql
contact_sellers = get_dataframe_from_sql(script)
contact_sellers['groups'] = 'beexp_seller'

In [ ]:
from dataclasses import dataclass
from typing import List

@dataclass
class Contact:

    id: str
    name: str
    email: str
    phone: str
    groups: List[str]
contacts = contact_sellers.apply(lambda row: Contact(**row), axis=1)

In [ ]:
for contact in contacts:
    contact.groups = [contact.groups]

In [ ]:
from messages import get_contact_by_id, get_sellers_contacts_from_database, generate_message_by_template, send_message_by_messenger, send_message_by_all_messengers, send_message_by_group, send_message_by_seller_id

sellers_contacts = get_sellers_contacts_from_database()

In [ ]:
from messages import get_contacts_from_json, send_message_by_group
contacts = get_contacts_from_json('messages/contacts.json')
send_message_by_group(
    'beexp_manager',
    'test',
    {'subject': 'Vendas BEEXP', 'beexp_order': 'teste: t, data: d, tudo ok'},
    contacts
)